In [5]:
!pip install Langchain
!pip install Transformers
!pip install groq langchain_groq

In [18]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    """
    ### Instruction
    You are a helpful assistant. When provided with a country or city name, respond with a concise and informative description about it in 3-4 sentences.
    
    Name: {name}
    """
)


In [14]:
from langchain_groq import ChatGroq
model=ChatGroq(
    temperature=0,
    model_name='llama-3.1-70b-versatile',
    groq_api_key=''
)

In [15]:
chain=model|prompt_template

In [19]:
res=chain.invoke("rome")

In [20]:
print(res)

text='\n    content="Rome, the Eternal City. \\n\\nRome is the capital city of Italy and is known for its rich history, architecture, art, and culture. It\'s a city that has been inhabited for over 2,500 years, and its history is still visible in its many ancient ruins, landmarks, and museums.\\n\\nSome of the most famous landmarks in Rome include:\\n\\n1. **The Colosseum**: This ancient amphitheater is one of Rome\'s most iconic symbols and a testament to the city\'s rich history.\\n2. **The Vatican City**: The center of the Catholic Church, the Vatican is home to numerous iconic landmarks, including St. Peter\'s Basilica, the Sistine Chapel, and the Vatican Museums.\\n3. **The Roman Forum**: A sprawling archaeological site that features ancient ruins, temples, and basilicas.\\n4. **The Pantheon**: A magnificently preserved ancient temple with an impressive dome.\\n5. **The Trevi Fountain**: A beautiful baroque fountain where visitors can make a wish by throwing a coin into the water.

In [21]:
from langchain.chains import LLMChain
llm=LLMChain(llm=model,prompt=prompt_template)

<ipython-input-21-0b61381a772d>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm=LLMChain(llm=model,prompt=prompt_template)


In [22]:
llm.invoke('rome')

{'name': 'rome',
 'text': "Rome, the capital of Italy, is a city steeped in history and architecture, with landmarks like the Colosseum and the Pantheon showcasing its rich past. The city is home to numerous iconic sites, including the Vatican City, which is an independent city-state within Rome and features famous attractions like the Sistine Chapel and St. Peter's Basilica. Rome's historic center is a UNESCO World Heritage Site, and the city is known for its delicious Italian cuisine, charming piazzas, and vibrant cultural scene. From ancient ruins to world-class museums, Rome has something to offer for every kind of traveler."}

In [24]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
llm=LLMChain(llm=model,prompt=prompt_template_name)
print(llm.run("Mexican"))

Here are some fancy name suggestions for your Mexican restaurant:

1. **Fiesta Royale**: This name evokes the vibrant and lively atmosphere of a Mexican fiesta, with a touch of luxury and sophistication.
2. **El Jardín de las Flores**: This name, which translates to "The Garden of Flowers," suggests a beautiful and elegant dining experience, with a nod to Mexico's rich cultural heritage.
3. **Casa de Sol**: This name, which means "House of the Sun," conveys a sense of warmth and hospitality, perfect for a restaurant that serves up delicious Mexican cuisine.
4. **La Hacienda**: This name, which means "The Estate," implies a sense of grandeur and luxury, while also evoking the traditional Mexican haciendas that are steeped in history and culture.
5. **Tres Amigos**: This name, which means "Three Friends," suggests a warm and welcoming atmosphere, perfect for a restaurant that wants to make its customers feel like part of the family.
6. **Mexiko**: This name is short and simple, yet elega

In [26]:
from langchain.chains import SimpleSequentialChain
prompt_template_name = PromptTemplate(
    input_variables =['country'],
    template = "I want you to return the capital of the {country}"
)

name_chain =LLMChain(llm=model, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['captal'],
    template="""Suggest one best line for this {capital}"""
)

food_items_chain = LLMChain(llm=model, prompt=prompt_template_items)

In [29]:
chain=SimpleSequentialChain(chains=[name_chain,food_items_chain])
res=chain.run("Pakistan")
print(res)

Located in the northwest of the country, Islamabad is a well-planned city that serves as the administrative and political hub of Pakistan.


In [36]:
from langchain.chains import SequentialChain
prompt_template_name = PromptTemplate(
    input_variables =['country'],
    template = "I want you to return the capital of the {country}"
)

name_chain =LLMChain(llm=model, prompt=prompt_template_name,output_key="capital name")

prompt_template_items = PromptTemplate(
    template="""Suggest one best line and best place for this."""
)

food_items_chain = LLMChain(llm=model, prompt=prompt_template_items,output_key="place")

In [38]:
chain=SequentialChain(chains=[name_chain,food_items_chain],
    input_variables = ['country'],
    output_variables = ['capital name', "place"])
res=chain({"country":"Pakistan"})
print(res)

{'country': 'Pakistan', 'capital name': 'The capital of Pakistan is Islamabad.', 'place': 'The line: "Life is a journey, not a destination." \nBest place: On a mountain summit, engraved on a stone or a plaque, overlooking a breathtaking view, to remind travelers of the importance of appreciating the journey.'}


In [42]:
!pip install wikipedia langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:0000:0100:01


In [43]:
from langchain.agents import AgentType, initialize_agent, load_tools
tools = load_tools(["wikipedia", "llm-math"], llm=model)

In [45]:
agent = initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("What was the GDP of US in 2024?")



> Entering new AgentExecutor chain...
To find the GDP of the US in 2024, I should first check if the data for 2024 is available on Wikipedia, as it is a reliable source for such information.

Action: wikipedia
Action Input: US GDP 2024
Observation: Page: List of U.S. states and territories by GDP
Summary: This is a list of U.S. states and territories by gross domestic product (GDP). This article presents the 50 U.S. states and the District of Columbia and their nominal GDP at current prices.
The data source for the list is the Bureau of Economic Analysis (BEA) in 2024. The BEA defined GDP by state as "the sum of value added from all industries in the state."
Nominal GDP does not take into account differences in the cost of living in different countries, and the results can vary greatly from one year to another based on fluctuations in the exchange rates of the country's currency. Such fluctuations may change a country's ranking from one year to the next, even though they often make l

'$29.017 trillion.'

In [46]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)
res= chain.run("Pakistan")
print(res)

<ipython-input-46-2d0ca68a35d0>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


Pakistan is a country located in South Asia, bordered by India, Afghanistan, Iran, and China. It has a diverse geography, ranging from the Himalayan mountains in the north to the Arabian Sea in the south, and is home to a rich cultural heritage with a blend of Islamic, Sikh, and Buddhist influences. The country has a population of over 220 million people, with major cities including Karachi, Lahore, and Islamabad, the capital. Pakistan is known for its vibrant cities, historic landmarks, and natural beauty, including the Karakoram Highway and the ancient Indus Valley Civilization.


In [47]:
chain.run("UAE")

'The United Arab Emirates (UAE) is a federation of seven emirates located on the eastern coast of the Arabian Peninsula. Known for its modern architecture, luxurious lifestyle, and rich cultural heritage, the UAE is a popular tourist destination, with cities like Dubai and Abu Dhabi offering world-class amenities and attractions. The country is also home to a diverse economy, with a strong focus on oil and gas, finance, and trade. From its stunning desert landscapes to its vibrant cityscapes, the UAE offers a unique blend of traditional and modern experiences.'

In [49]:
chain.memory.buffer

'Human: Pakistan\nAI: Pakistan is a country located in South Asia, bordered by India, Afghanistan, Iran, and China. It has a diverse geography, ranging from the Himalayan mountains in the north to the Arabian Sea in the south, and is home to a rich cultural heritage with a blend of Islamic, Sikh, and Buddhist influences. The country has a population of over 220 million people, with major cities including Karachi, Lahore, and Islamabad, the capital. Pakistan is known for its vibrant cities, historic landmarks, and natural beauty, including the Karakoram Highway and the ancient Indus Valley Civilization.\nHuman: UAE\nAI: The United Arab Emirates (UAE) is a federation of seven emirates located on the eastern coast of the Arabian Peninsula. Known for its modern architecture, luxurious lifestyle, and rich cultural heritage, the UAE is a popular tourist destination, with cities like Dubai and Abu Dhabi offering world-class amenities and attractions. The country is also home to a diverse econ

In [50]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=model)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<ipython-input-50-773109720b9b>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=model)


In [52]:
convo.run("Who won the 1992 cricket worldcup ")

"The 1992 Cricket World Cup was won by Pakistan. The tournament was held in Australia and New Zealand from February 22 to March 25, 1992. Pakistan, led by Imran Khan, defeated England in the final by 22 runs at the Melbourne Cricket Ground. The final score was Pakistan 249/6 in 50 overs, and England 227 in 49.2 overs. It was a historic win for Pakistan, and it marked their first Cricket World Cup title. If you're interested, I can provide more information about the tournament, such as the top run-scorers, wicket-takers, or notable matches. Would you like that?"

In [53]:
convo.run("Write python program to add 2 numbers")

'Here\'s a simple Python program that adds two numbers:\n\n```python\ndef add_numbers(num1, num2):\n    """\n    This function takes two numbers as input and returns their sum.\n    \n    Parameters:\n    num1 (int or float): The first number to be added.\n    num2 (int or float): The second number to be added.\n    \n    Returns:\n    int or float: The sum of num1 and num2.\n    """\n    return num1 + num2\n\n# Example usage:\nnum1 = 5\nnum2 = 7\nresult = add_numbers(num1, num2)\nprint(f"The sum of {num1} and {num2} is: {result}")\n```\n\nWhen you run this program, it will output: `The sum of 5 and 7 is: 12`. You can replace `num1` and `num2` with any two numbers you want to add.\n\nIf you want to make the program more interactive, you can use the `input()` function to get the numbers from the user:\n\n```python\ndef add_numbers(num1, num2):\n    return num1 + num2\n\nnum1 = float(input("Enter the first number: "))\nnum2 = float(input("Enter the second number: "))\nresult = add_number

In [54]:
convo.run("who was the captain of the 1992 world cup")

"The 1992 Cricket World Cup, which was won by Pakistan, had Imran Khan as its captain. Imran Khan is a legendary Pakistani cricketer and politician who led the team to their first Cricket World Cup title in 1992. He is widely regarded as one of the greatest fast bowlers and all-rounders in the history of cricket. Under his captaincy, Pakistan defeated England in the final at the Melbourne Cricket Ground on March 25, 1992. Imran Khan's leadership and performance were instrumental in Pakistan's victory, and he is still remembered as a hero in Pakistan for his role in the team's success. Would you like to know more about Imran Khan's cricket career or his achievements as a captain?"

In [55]:
convo.memory.buffer

'Human: Who won the 1992 worldcup\nAI: The 1992 World Cup did not actually take place. The World Cup is held every 4 years, and the tournaments surrounding that year were the 1990 FIFA World Cup in Italy, which was won by West Germany, and the 1994 FIFA World Cup in the United States, which was won by Brazil. If you\'re interested, I can provide more information about those tournaments, such as the final scores, notable players, or memorable moments. Would you like that?\nHuman: Who won the 1992 cricket worldcup \nAI: The 1992 Cricket World Cup was won by Pakistan. The tournament was held in Australia and New Zealand from February 22 to March 25, 1992. Pakistan, led by Imran Khan, defeated England in the final by 22 runs at the Melbourne Cricket Ground. The final score was Pakistan 249/6 in 50 overs, and England 227 in 49.2 overs. It was a historic win for Pakistan, and it marked their first Cricket World Cup title. If you\'re interested, I can provide more information about the tourna

In [56]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm=model,
    memory=memory
)
convo.run("Who won the first cricket world cup?")

<ipython-input-56-dff773439e41>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3)


"The first Cricket World Cup was held in 1975, and it was won by the West Indies. The tournament was hosted by England, and the final match was played at Lord's Cricket Ground in London on June 21, 1975. The West Indies team, led by Clive Lloyd, defeated Australia by 17 runs in the final, with a score of 291/8 in 60 overs, while Australia managed 274 in 58.4 overs. The West Indies' victory was a significant moment in cricket history, marking the beginning of their dominance in international cricket during the late 1970s and early 1980s. Would you like to know more about the 1975 Cricket World Cup or the West Indies team's performance during that era?"

In [57]:
convo.run("what is 5+5")

"The answer to 5+5 is 10. This is a basic arithmetic operation that involves adding two numbers together. In this case, when you add 5 and 5, the result is 10. This type of calculation is a fundamental concept in mathematics and is used in a wide range of applications, from simple everyday calculations to complex mathematical equations. If you'd like to explore more math concepts or practice calculations, I'd be happy to help. Alternatively, we could go back to discussing cricket or move on to a different topic altogether. What would you like to talk about next?"

In [58]:
convo.run("what is 5+3")

"The answer to 5+3 is 8. This is another basic arithmetic operation that involves adding two numbers together. In this case, when you add 5 and 3, the result is 8. It's interesting to note that arithmetic operations like this one are the building blocks of mathematics, and mastering them is essential for more complex calculations and problem-solving. For example, understanding how to add and subtract numbers is crucial for algebra, geometry, and other advanced math subjects. If you'd like to practice more calculations or explore different math concepts, such as multiplication, division, or fractions, I'd be happy to assist you. Alternatively, we could discuss how math is used in real-world applications, such as science, finance, or engineering. What would you like to talk about next?"

In [59]:
convo.run("what is 3+5")

"The answer to 3+5 is 8. This is a straightforward arithmetic operation that involves adding two numbers together. In this case, when you add 3 and 5, the result is indeed 8. It's worth noting that the order of the numbers doesn't change the result, so 3+5 is equal to 5+3, which we discussed earlier. This property of arithmetic is known as the commutative property of addition, and it's a fundamental concept in mathematics. If you'd like to explore more math concepts, such as the commutative property of multiplication or the distributive property, I'd be happy to explain them in more detail. Alternatively, we could practice more calculations or discuss how math is used in different areas, such as music, art, or computer science. What would you like to talk about next?"

In [62]:
convo.memory.buffer

"Human: what is 5+5\nAI: The answer to 5+5 is 10. This is a basic arithmetic operation that involves adding two numbers together. In this case, when you add 5 and 5, the result is 10. This type of calculation is a fundamental concept in mathematics and is used in a wide range of applications, from simple everyday calculations to complex mathematical equations. If you'd like to explore more math concepts or practice calculations, I'd be happy to help. Alternatively, we could go back to discussing cricket or move on to a different topic altogether. What would you like to talk about next?\nHuman: what is 5+3\nAI: The answer to 5+3 is 8. This is another basic arithmetic operation that involves adding two numbers together. In this case, when you add 5 and 3, the result is 8. It's interesting to note that arithmetic operations like this one are the building blocks of mathematics, and mastering them is essential for more complex calculations and problem-solving. For example, understanding how

In [63]:
convo.run("who was the captain")

"I don't know who the captain you're referring to is. We didn't discuss any specific team, sport, or context that would indicate who the captain might be. Earlier in our conversation, I mentioned cricket, but we didn't explore that topic further. If you'd like to discuss cricket or any other sport, I'd be happy to try and help. Alternatively, if you could provide more context or clarify which captain you're asking about, I'll do my best to provide a helpful response. What would you like to talk about next?"